# Correspondence-driven plane-based M3C2 (PB_M3C2)

In [ ]:
import py4dgeo

In [ ]:
py4dgeo.set_interactive_backend("vtk")

In [ ]:
py4dgeo.ensure_test_data_availability()

In [ ]:
epoch0, epoch1 = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)

In [ ]:
alg = py4dgeo.PB_M3C2()

In [ ]:
(
    xyz_epoch0,
    xyz_epoch1,
    extracted_segments,
) = alg.export_segmented_point_cloud_and_segments(
    epoch0=epoch0,
    epoch1=epoch1,
)

In [ ]:
# -----

In [ ]:
extended_y_with_label_1 = py4dgeo.generate_random_extended_y(
    extracted_segments, extended_y_file_name="extended_y.csv", low=1, high=1
)

In [ ]:
new_extended_y = py4dgeo.add_no_corresponding_seg(
    segments=extracted_segments,
    # extended_y=extended_y_with_label_1,
    threshold_max_distance=5,
    algorithm="random",
    extended_y_file_name="extended_y.csv",
)

In [ ]:
# -----

In [ ]:
alg.training(
    extracted_segments_file_name="extracted_segments.seg",
    extended_y_file_name="testdata-labelling.csv",
)

In [ ]:
distances, uncertainties = alg.compute_distances(epoch0=epoch0, epoch1=epoch1)